<a href="https://colab.research.google.com/github/AnshulH/NLP-DL-Group2/blob/lstm/NLP_Project_1_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!rm -r /content/NLP-DL-Group2/
!cd /content && git clone https://github.com/AnshulH/NLP-DL-Group2

Cloning into 'NLP-DL-Group2'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 66 (delta 6), reused 6 (delta 1), pack-reused 46
Unpacking objects: 100% (66/66), 30.21 MiB | 2.25 MiB/s, done.
Updating files: 100% (25/25), done.


In [18]:
def decode(vocab,corpus):
    
    text = ''
    for i in range(len(corpus)):
        wID = corpus[i]
        text = text + vocab[wID] + ' '
    return(text)

def encode(words,text):
    corpus = []
    tokens = text.split(' ')
    for t in tokens:
        try:
            wID = words[t][0]
        except:
            wID = words['<unk>'][0]
        corpus.append(wID)
    return(corpus)

def read_encode(file_name,vocab,words,corpus,threshold):
    
    wID = len(vocab)
    
    if threshold > -1:
        with open(file_name,'rt') as f:
            for line in f:
                line = line.replace('\n','')
                tokens = line.split(' ')
                for t in tokens:
                    try:
                        elem = words[t]
                    except:
                        elem = [wID,0]
                        vocab.append(t)
                        wID = wID + 1
                    elem[1] = elem[1] + 1
                    words[t] = elem

        temp = words
        words = {}
        vocab = []
        wID = 0
        words['<unk>'] = [wID,100]
        wID = 1
        words['<pad>'] = [wID,100]
        vocab.append('<unk>')
        vocab.append('<pad>')
        for t in temp:
            if temp[t][1] >= threshold:
                vocab.append(t)
                wID = wID + 1
                words[t] = [wID,temp[t][1]]
            
                    
    with open(file_name,'rt') as f:
        for line in f:
            line = line.replace('\n','')
            tokens = line.split(' ')
            for t in tokens:
                try:
                    wID = words[t][0]
                except:
                    wID = words['<unk>'][0]
                corpus.append(wID)
                
    return [vocab,words,corpus]

In [25]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words # added for detecting English words
import string
import nltk
import calendar
import re

nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
stop = set(stopwords.words('english') + list(string.punctuation))

In [22]:
# create a list of tuples containing a list with the tokenized bio and a fake/real label
# split_bios = [([bio1, tokens1],[REAL]),([bio2, tokens2],[FAKE])]

original_file = '/content/NLP-DL-Group2/hw#1/mix.train.txt'

with open(original_file) as input:
  all_text = input.readlines()

split_bios = []
line_text = []

# seq_status = 0 marks the start of the bio
# seq_status = 1 is the main text of the bio
# seq_status = 2 is the end of the bio
for line in all_text:
  line = line.strip()
  if "<start_bio>" in line:
    seq_status = 0
    continue
  if "<end_bio>" in line:
    seq_status = 2
    continue
  if seq_status == 0:
    bio_person = line.strip(" = ").lower().split()
    stop.update(bio_person) # could remove important words if name is meaningful
    seq_status = 1
    continue
  if seq_status == 1:
    line = re.sub("[\d-]", '',line)
    line = [token for token in word_tokenize(line.lower()) if token not in stop]
    if len(line) > 0:
      line_text.extend(line)
  if seq_status == 2 and '[' in line:
    stop -= set(bio_person)
    if line == '[FAKE]': #updated code so now fake / real is binary
      line = 0
    else:
      line = 1
    split_bios.append((line_text,line))
    line_text = []

# print(split_bios[:2])

In [23]:
encodings = read_encode('/content/NLP-DL-Group2/hw#1/mix.train.tok',[],{},[],3)
word_dict = encodings[1]
vocab_length = max(encodings[2])

bio_array = []
seq_len = 512
y = []
for bio in split_bios:
  wid = []
  for token in bio[0]:
    word_info = word_dict.get(token)
    if word_info is None:
      wid.append(0)
    else:
      wid.append(word_info[0])
  if len(wid) > seq_len:
    wid = wid[:seq_len]
  elif len(wid) < seq_len:
    pad_size = seq_len - len(wid)
    wid.extend(np.ones(pad_size,dtype=int))
  bio_array.append(np.array(wid))
  y.append(bio[1])

list_len = [len(i) for i in bio_array]
# print(bio_array[:5])
# print(max(list_len))
dummy = torch.tensor(bio_array)
y = torch.tensor(y)
# print(dummy)
# print(y)

In [ ]:
np.ones(4,dtype=int)

array([1, 1, 1, 1])

In [26]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class LSTM(nn.Module):
    def __init__(self,vocab,words,d_model,d_hidden,n_layers,dropout_rate):
        super().__init__()  
        self.vocab = vocab
        self.words = words
        self.vocab_size = len(self.vocab)
        self.n_layers = n_layers
        self.d_hidden = d_hidden
        self.d_model = d_model
        self.embedding = nn.Embedding(self.vocab_size,d_model)
        #self.lstm = nn.LSTM(input_size=300,hidden_size=d_hidden,num_layers=1,batch_first=True,bidirectional=True)
        self.drop = dropout_rate
        self.fc = nn.Linear(d_hidden, 1) 
        self.lstm = nn.LSTM(
            input_size=d_model,
            hidden_size=d_hidden,
            num_layers=n_layers,
            dropout=dropout_rate
        ) 
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.lin = nn.Linear(512, 1) 
        
    def forward(self,x,prev_state):
        embed = self.embedding(x)
        print(embed.size())
        print("k")
        print(prev_state[0].size())
        print(prev_state[1].size())
        batch_size = x.size(0)
        print(batch_size)
        output, state = self.lstm(embed, self.detach_hidden(prev_state))
        print("dha")
        print(output.size()) 
        print(state[0].size())
        print(state[1].size()) 
        logits = self.fc(output)
        print(logits.size())
        out = self.relu(logits)
        print(out.size())
        out = out.view(batch_size, -1)
        print(out.size()) 
        out = self.lin(out)
        out = self.sigmoid(out)
        print(out.size())
        return out, state 
        # return [preds,h]
    
    def init_weights(self,sequence_length): 
        return (torch.zeros(self.n_layers, 512, self.d_hidden),
                torch.zeros(self.n_layers, 512, self.d_hidden))
    
    def detach_hidden(self, hidden):
        h, c = hidden
        h = h.detach()
        c = c.detach()
        return (h,c)
       

        
    

In [27]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import math
import time
import numpy as np
import sys
import argparse
import os

device = "cpu"



class FFNN(nn.Module):
    def __init__(self, vocab, words,d_model, d_hidden, dropout):
        super().__init__() 
    
        self.vocab = vocab
        self.words = words
        self.vocab_size = len(self.vocab)
        self.d_model = d_model
        self.d_hidden = d_hidden
        self.dropout = dropout
        self.embeds = nn.Embedding(self.vocab_size,d_model)
#          {perform other initializations needed for the FFNN}

    def forward(self, src):
        embeds = self.dropout(self.embeds(src))
#          {add code to implement the FFNN}
        pass
        # return x
                
    def init_weights(self):
      pass
#          {perform initializations}
             
# class LSTM(nn.Module):
#     def __init__(self,vocab,words,d_model,d_hidden,n_layers,dropout_rate):
#         super().__init__()
        
#         self.vocab = vocab
#         self.words = words
#         self.vocab_size = len(self.vocab)
#         self.n_layers = n_layers
#         self.d_hidden = d_hidden
#         self.d_model = d_model
#         self.embeds = nn.Embedding(self.vocab_size,d_model)
# #          {perform other initializations needed for the LSTM}
#         self.lstm = nn.LSTM(d_model, d_hidden, n_layers, dropout=dropout_rate, batch_first=True)
#         self.dropout = nn.Dropout(dropout_rate)
#         self.fc = nn.Linear(d_hidden, 1)
#         self.sigmoid = nn.Sigmoid()
        
#     def forward(self,src,h):
#         embeds = self.dropout(self.embeds(src))
#         batch_size = src.size(0)
#         src = src.long()
#         lstm_out, h = self.lstm(embeds, h)
#         lstm_out = lstm_out.contiguous().view(-1, self.d_hidden)
        
#         out = self.dropout(lstm_out)
#         out = self.fc(out)
#         out = self.sigmoid(out)
        
#         out = out.view(batch_size, -1)
#         out = out[:,-1]
#         return out, h
    
#     def init_weights(self, batch_size):
#         weight = next(self.parameters()).data
#         hidden = (weight.new(self.n_layers, batch_size, self.d_hidden).zero_().to(device),
#                       weight.new(self.n_layers, batch_size, self.d_hidden).zero_().to(device))
#         return hidden
        
    
#     def detach_hidden(self, hidden):
#       pass
# #          {needed for training...}
#         # return [hidden, cell]  
 
def main():
    
    parser = argparse.ArgumentParser()
    parser.add_argument('-d_model', type=int, default=100)
    parser.add_argument('-d_hidden', type=int, default=100)
    parser.add_argument('-n_layers', type=int, default=2)
    parser.add_argument('-batch_size', type=int, default=20)
    parser.add_argument('-seq_len', type=int, default=30)
    parser.add_argument('-printevery', type=int, default=5000)
    parser.add_argument('-window', type=int, default=3)
    parser.add_argument('-epochs', type=int, default=20)
    parser.add_argument('-lr', type=float, default=0.0001)
    parser.add_argument('-dropout', type=int, default=0.35)
    parser.add_argument('-clip', type=int, default=2.0)
    parser.add_argument('-model', type=str,default='LSTM')
    parser.add_argument('-savename', type=str,default='lstm')
    parser.add_argument('-loadname', type=str)
    parser.add_argument('-trainname', type=str,default='wiki.train.txt')
    parser.add_argument('-validname', type=str,default='wiki.valid.txt')
    parser.add_argument('-testname', type=str,default='wiki.test.txt')

    params = {
        'd_model': 1,
        'd_hidden': 1,
        'n_layers': 1,
        'batch_size': 100,
        'seq_len': 512,
        'printevery': 5000,
        'window': 3,
        'epochs': 20,
        'lr': 0.0001,
        'dropout': 0.35,
        'clip': 2.0,
        'model': 'LSTM',
        'savename': 'lstm',
        'loadname': None,
        'trainname': '/content/NLP-DL-Group2/hw#1/mix.train.tok',
        'validname': '/content/NLP-DL-Group2/hw#1/mix.train.tok',
        'testname': '/content/NLP-DL-Group2/hw#1/mix.train.tok'
    }
    parser.add_argument("-f", required=False)
    
    # params = parser.parse_args()    
    # torch.manual_seed(0)
    
    [vocab,words,train] = read_encode(params['trainname'],[],{},[],3)
    print('vocab: %d train: %d' % (len(vocab),len(train)))
    [vocab,words,test] = read_encode(params['testname'],vocab,words,[],-1)
    print('vocab: %d test: %d' % (len(vocab),len(test)))
    params['vocab_size'] = len(vocab)

    train_loader = read_encode(params['trainname'],[],{},[],3)
    
    if params['model'] == 'FFNN':
      pass
#          {add code to instantiate the model, train for K epochs and save model to disk}
        
    if params['model'] == 'LSTM':
      model = LSTM(vocab,words,params['d_model'],params['d_hidden'],params['n_layers'],params['dropout']) 
      model.to(device)

      # lr=0.005
      # criterion = nn.BCELoss()
      # optimizer = torch.optim.Adam(model.parameters(), lr=lr)

      # epochs = 2
      # counter = 0
      # print_every = 1000
      # clip = 5
      # valid_loss_min = np.Inf

      # model.train()
      # print("dha")
      # print(list(train_loader[1].items())[0:15])

      # for i in range(epochs):
      #   h = model.init_weights(params['batch_size'])
        
      #   for vocab, labels in oops:
      #       counter += 1
      #       h = tuple([e.data for e in h])
      #       vocab, labels = vocab.to(device), labels.to(device)
      #       model.zero_grad()
      #       output, h = model(vocab, h)
      #       loss = criterion(output.squeeze(), labels.float())
      #       loss.backward()
      #       nn.utils.clip_grad_norm_(model.parameters(), clip)
      #       optimizer.step()
            
      #       if counter%print_every == 0:
      #           val_h = model.init_hidden(read_encode(params['validname'],[],{},[],3))
      #           val_losses = []
      #           model.eval()
      #           for inp, lab in params['validname']:
      #               val_h = tuple([each.data for each in val_h])
      #               inp, lab = inp.to(device), lab.to(device)
      #               out, val_h = model(inp, val_h)
      #               val_loss = criterion(out.squeeze(), lab.float())
      #               val_losses.append(val_loss.item())
                    
      #           model.train()
      #           print("Epoch: {}/{}...".format(i+1, epochs),
      #                 "Step: {}...".format(counter),
      #                 "Loss: {:.6f}...".format(loss.item()),
      #                 "Val Loss: {:.6f}".format(np.mean(val_losses)))
      #           if np.mean(val_losses) <= valid_loss_min:
      #               torch.save(model.state_dict(), './state_dict.pt')
      #               print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
      #               valid_loss_min = np.mean(val_losses) 
      criterion = nn.BCEWithLogitsLoss()
      optimizer = torch.optim.Adam(model.parameters(), lr=0.5) 

      for epoch in range(20):  
              state_h, state_c = model.init_weights(512)
              print(state_h.size())
              optimizer.zero_grad()

              y_pred, (state_h, state_c) = model(dummy[:500], (state_h, state_c))
              print(y[:2].size())
              print("j")
              print(y_pred.view(-1).size())
              print(y_pred.type())

              w = y.type(torch.FloatTensor)
              print(w[:2].type()) 
              loss = criterion(y_pred.view(-1), w[:500])

              state_h = state_h.detach()
              state_c = state_c.detach()

              loss.backward()
              optimizer.step()
              print({ 'epoch': epoch, 'batch': params['batch_size'], 'loss': loss.item() })


    if params['model'] == 'FFNN_CLASSIFY':
      pass
#          {add code to instantiate the model, recall model parameters and perform/learn classification}

    if params['model'] == 'LSTM_CLASSIFY':
      pass
#          {add code to instantiate the model, recall model parameters and perform/learn classification}
        
    print(params)
    

if __name__ == "__main__":
    main()

vocab: 35151 train: 3012820
vocab: 35151 test: 3012820


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.35 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


torch.Size([1, 512, 1])
torch.Size([500, 512, 1])
k
torch.Size([1, 512, 1])
torch.Size([1, 512, 1])
500
dha
torch.Size([500, 512, 1])
torch.Size([1, 512, 1])
torch.Size([1, 512, 1])
torch.Size([500, 512, 1])
torch.Size([500, 512, 1])
torch.Size([500, 512])
torch.Size([500, 1])
torch.Size([2])
j
torch.Size([500])
torch.FloatTensor
torch.FloatTensor
{'epoch': 0, 'batch': 100, 'loss': 0.7222668528556824}
torch.Size([1, 512, 1])
torch.Size([500, 512, 1])
k
torch.Size([1, 512, 1])
torch.Size([1, 512, 1])
500
dha
torch.Size([500, 512, 1])
torch.Size([1, 512, 1])
torch.Size([1, 512, 1])
torch.Size([500, 512, 1])
torch.Size([500, 512, 1])
torch.Size([500, 512])
torch.Size([500, 1])
torch.Size([2])
j
torch.Size([500])
torch.FloatTensor
torch.FloatTensor
{'epoch': 1, 'batch': 100, 'loss': 0.709483802318573}
torch.Size([1, 512, 1])
torch.Size([500, 512, 1])
k
torch.Size([1, 512, 1])
torch.Size([1, 512, 1])
500
dha
torch.Size([500, 512, 1])
torch.Size([1, 512, 1])
torch.Size([1, 512, 1])
torch.Siz